In [51]:
import tensorflow as tf
import warnings

warnings.filterwarnings('ignore')
print(tf.__version__)

2.10.0


In [54]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

In [55]:
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'
VAL_DIR = 'data/val'

In [56]:
train_datagen = ImageDataGenerator(
                                rescale=1/255,
                                rotation_range=20,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest'

                                   )
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size=(300,300),
                                                    batch_size=10,
                                                    class_mode='categorical',
                                                    color_mode='grayscale')

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                    target_size=(300,300),
                                                    batch_size=10,
                                                    class_mode='categorical',
                                                    color_mode='grayscale')

Found 5216 images belonging to 3 classes.
Found 624 images belonging to 3 classes.


In [57]:
# model to predict 3 classes
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='sigmoid')
])

In [59]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 298, 298, 16)      160       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 149, 149, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 35, 35, 64)      

In [60]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='acc', patience=2)

In [61]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['acc'])

In [62]:
model.fit(train_generator,
          epochs=20,
          validation_data=test_generator,
          callbacks=[early_stop])

Epoch 1/20
522/522 [==============================] - 171s 327ms/step - loss: 0.8447 - acc: 0.6198 - val_loss: 0.8238 - val_acc: 0.6330
Epoch 2/20
522/522 [==============================] - 180s 344ms/step - loss: 0.7228 - acc: 0.6929 - val_loss: 0.7067 - val_acc: 0.7468
Epoch 3/20
522/522 [==============================] - 178s 341ms/step - loss: 0.6792 - acc: 0.7132 - val_loss: 0.6589 - val_acc: 0.7532
Epoch 4/20
522/522 [==============================] - 169s 324ms/step - loss: 0.6700 - acc: 0.7189 - val_loss: 0.7007 - val_acc: 0.6571
Epoch 5/20
522/522 [==============================] - 182s 348ms/step - loss: 0.6439 - acc: 0.7320 - val_loss: 0.7608 - val_acc: 0.7244
Epoch 6/20
522/522 [==============================] - 169s 325ms/step - loss: 0.6307 - acc: 0.7327 - val_loss: 1.1659 - val_acc: 0.5673
Epoch 7/20
522/522 [==============================] - 166s 318ms/step - loss: 0.6229 - acc: 0.7372 - val_loss: 0.7291 - val_acc: 0.8061
Epoch 8/20
522/522 [============================

In [63]:
model.save('model_categorical.h5')

In [64]:
# evaluate
loss, acc = model.evaluate(test_generator)
print('loss: ', loss)
print('acc: ', acc)

63/63 [==============================] - 4s 64ms/step - loss: 0.8181 - acc: 0.7388
loss:  0.8181131482124329
acc:  0.7387820482254028


In [43]:
model

In [42]:
model = tf.keras.models.load_model('model/model_categorical.h5')

ValueError: The channel dimension of the inputs should be defined. The input_shape received is (None, None, None, None), where axis -1 (0-based) is the channel dimension, which found to be `None`.

In [65]:
# predict
import numpy as np
from tensorflow.keras.preprocessing import image

img = image.load_img('data/train/PNEUMONIA VIRUS/person86_virus_159.jpeg', target_size=(300,300))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

print(img.shape)

(1, 300, 300, 3)


In [93]:
IMG_PATH = [
    'data/test/NORMAL/IM-0033-0001-0002.jpeg',
    'data/test/PNEUMONIA BACTERIA/person80_bacteria_389.jpeg'
]

In [112]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [148]:
# Load the image
img_path = 'data/test/PNEUMONIA BACTERIA/person80_bacteria_389.jpeg'
img = load_img(img_path, target_size=(300, 300), color_mode='grayscale')

# Convert it to a numpy array with target shape
img = img_to_array(img)

# Rescale by 1/255
img = img/255

# Add a fourth dimension
img = np.expand_dims(img, axis=0)


In [149]:
classes = model.predict(img, batch_size=10)
print(classes)

1/1 [==============================] - 0s 40ms/step
[[0.03133188 0.91333777 0.7687464 ]]


In [150]:
classes = np.argmax(classes, axis=1)

In [151]:
classes.astype(int)

array([1])